<a href="https://colab.research.google.com/github/Ayancodes2601/AudioTranscriber/blob/main/Audio_Transcriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

In [ ]:
!whisper "/content/drive/MyDrive/Lecture 2 _ Image Classification.mp4" --model medium.en

In [ ]:
import re
import json

def group_complete_thoughts(timestamps):
    grouped_timestamps = []
    current_group = []

    for start_time, text in timestamps:
        current_group.append(text)

        if text[-1] in ['.', '!', '?'] and len(current_group) >= 3:
            grouped_timestamps.append((start_time, " ".join(current_group)))
            current_group = []

    return grouped_timestamps

def format_time(time_str):
    h, m, s = map(int, re.sub(r',\d{3}', '', time_str).split(':'))
    return f"{h:02d}:{m:02d}:{s:02d}"

def srt_to_json(srt_content):
    json_data = {
        "dataset_link": "https://www.youtube.com/watch?v=OoUX-nOEjG0&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=2",
        "dataset": {}
    }

    pattern = r"\d+\n(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.+?)(?=\n\d+\n\d{2}:\d{2}:\d{2},\d{3} -->|\Z)"
    matches = re.findall(pattern, srt_content, re.DOTALL)

    timestamps = [(format_time(match[0]), match[2].strip()) for match in matches]
    grouped_timestamps = group_complete_thoughts(timestamps)

    for i, (start_time, combined_text) in enumerate(grouped_timestamps):
        json_data["dataset"][start_time] = combined_text

    return json.dumps(json_data, indent=2)

# Replace this with the actual path to your SRT file
srt_file_path = "/content/Lecture 2 _ Image Classification.srt"
with open(srt_file_path, "r") as srt_file:
    srt_content = srt_file.read()
    json_data = srt_to_json(srt_content)

with open("transcript1.json", "w") as file:
    file.write(json_data)
